# Homework 6

<p style="text-align:right;color:red;font-weight:bold;font-size:16pt;padding-bottom:20px">Please, copy this notebook before editing!</p>

In [ ]:
%reload_ext autoreload
%autoreload 2
from typing import Set, List, Tuple, Any
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt

## Fraud Classification with Decision Trees

Review:
- Chapter 4 of the ML Book
- Scikit-Learn Library https://scikit-learn.org/

In [ ]:
categorical_features = ['InsuranceType', 'MaritalStatus', 'OvernightHospitalStay', 'InjuryType']
numerical_features = ['NumSoftTissue', 'NumClaimants', 'TotalClaimed', 'IncomeofPolicyHolder', 
                      'ClaimAmountReceived', 'PercentSoftTissue', 'NumClaims', 'ClaimAmount']
target_feature = 'FraudFlag'

### 6.1 Load Data, Split in Train and Test Sets
The insurance fraud data are available at `/data/kelleher/MotorInsuranceFraudClaim_clean.csv`

In this exercise you will use this dataset to train a decision tree classifier. The first step ofr any supervised machine learning task is to split the data into training and test sets.

Write a function `split_data_set` that randomly splits a given dataframe into two frames to a given fraction. E.g. a value of $0.8$ should give 80% of the rows to the first dataframe, and 20% to the second.

Save your solution to `solution_6_1.py`

In [ ]:
%%writefile solution_6_1.py
from typing import Set, List, Tuple, Any
import pandas as pd

def split_data_set(df: pd.DataFrame, frac) -> Tuple[pd.DataFrame, pd.DataFrame]:
### your code here
    




    

In [ ]:
# Try your code
from solution_6_1 import split_data_set

data = '/data/kelleher/MotorInsuranceFraudClaim_clean.csv'
df = pd.read_csv(data)
print(f"Number of records: {df.shape[0]:,}")

df_train, df_test = split_data_set(df, 0.8)
print(f"Number of rows in Training-Set: {df_train.shape[0]:,}\nNumber of rows in Test-Set: {df_test.shape[0]:,}")

#### Test 6.1 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 1

### 6.2 One-Hot-Encoding
Many Python ML packages are designed to work with numerical data. We need to convert our categorical features into their on-hot-encoded representation.

Use the one-hot-encoder from the Scikit-Learn library
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

Define a function that creates (and fits) a one-hot-encoder for a given dataframe and a list of categorical features that the resulting encoder should transform. The function should return the encoder object that can be used for subsequent transformations.

**Note**: Read the documentation and configure the encode so that it:
1. handles unknown levels by ignoring them,
2. drop the firt the dimenson of the one-hot-encoded vector,
3. produces a **dense** output matrix, not a *sparse* matrix

Save your solution to `solution_6_2.py`

In [ ]:
%%writefile solution_6_2.py
from typing import Set, List, Tuple, Any
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def create_one_hot_encoder(df: pd.DataFrame, cat_features: List[str]) -> OneHotEncoder:
    ### your code here
  






    

In [ ]:
# Try your code
from solution_6_2 import create_one_hot_encoder

df_train, df_test = split_data_set(df, 0.8)
categorical_features = ['InsuranceType', 'MaritalStatus', 'OvernightHospitalStay', 'InjuryType']
my_encoder = create_one_hot_encoder(df_train, categorical_features)
display(my_encoder.categories_)
X_cat = my_encoder.transform(df_train[categorical_features].values)
print(X_cat.shape)
display(X_cat[:10,:])

#### Test 6.2 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 2

### 6.3 Build Matrices for Descriptive Features and Target

Combine the respective matrices of the categorical and numeric features to a matrix $X$ Create a vector $y$ for the target feature.
We do this separately for both: training and test sets.

Create a function that takes to feature matrices and concatenates them side-by-side. E.g. a matrix with one-hot-encoded categorical features and a matrix with the numerical featurs need to be combined to a single feature matrix.

Save your solution to `solution_6_3.py`

In [ ]:
%%writefile solution_6_3.py
import pandas as pd
import numpy as np

def concat_feature_matrix(mat_left: np.ndarray, mat_right: np.ndarray) -> np.ndarray:
    ### your code here




    

In [ ]:
# Try your code
import numpy as np
from solution_6_3 import concat_feature_matrix

X_train = concat_feature_matrix(
    my_encoder.transform(df_train[categorical_features].values),
    df_train[numerical_features].values
)
print(X_train.shape)

#### Test 6.3 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 3

### 6.4 Train Decision Tree

Let's train the model. You will use the `DecisionTreeClassifier` from the scikit-learn package https://scikit-learn.org/stable/modules/tree.html#tree

Complete these streps:
1. Split the original data into training and test sets with a 70:30 split (training is 70%)
2. Create feature matrices $X_{train}$ and $X_{test}$ from the respective data sets. These are matrices (np.ndarray) that have categorical data one-hot-encoded.
3. Create vectors $y_{train}$ and $y_{test}$ from the respective target column
4. Create a `DecisionTreeClassifier` classifier object and train it.


In [ ]:
from sklearn import tree

X_train_cat = # complete this expression
X_train_num = # complete this expression
X_train = # complete this expression
y_train = # complete this expression

### Train the model
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

We can review the decision tree

In [ ]:
tree.plot_tree(clf)

#### Test 6.4 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 4

### 6.5 Model Evaluation

You want to know how well the model performs on our test data. You prepare the test dataset the same way you prepared the training data: one-hot-encoding, then merge the feature matrices to one. Create a target vector $y_{test}$ with the actual results.

Then use the ML model to predict $\hat{y}$ from the $X_{test}$ data.

You can then count how many records were predicted correctly and incorrectly. Create a function that takes the actual target vector and the predicted target vector, and produces counts for:
- True Positive (TP): number of records that are actually TRUE and predicted as TRUE
- False Negative (FN): number of records that are actually TRUE and predicted as FALSE
- False Positive (FP): number of records that are actually FALSE and predicted as TRUE
- True Negative (TN): number of records that are actually FALSE and predicted as FALSE

Note: In this dataset 0 = False, and 1 = True

Save your solution to `solution_6_5.py`

In [ ]:
%%writefile solution_6_5.py
from typing import Tuple
import numpy as np

def evaluate_classification(y_actual: np.ndarray, y_predicted: np.ndarray) -> Tuple[int, int, int, int]:
    ### your code here




In [ ]:
# Try your code
from solution_6_5 import evaluate_classification

# 1. Prepare the Test Data
X_test_cat = # ....
X_test_num = # ....
X_test = # ....
y_test = # ....

# 1. run inference of the classifier on out test set
y_hat = clf.predict(X_test)

# 2. compare the actual (y_test) with the predicted (y_hat)
TP, FN, FP, TN = evaluate_classification(y_test, y_hat)

# 3. Review the results. This is called a "confusion matrix"
print(f"""
              | Predicted
              | True  False
--------------+------------------
Actual True   | {TP:4d}   {FN:4d}
       False  | {FN:4d}   {TN:4d}
""")


#### Test 6.5 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 5

# TestingExecute the cell below to run all tests...

In [ ]:
! test/run_test.sh

# Homework Submission
- This homework is due by March 27, 2024, 5:30PM (EDT)
- Make sure that all your programs and output files are in the exact folder as specified in the instructions.
- All file names on this system are case sensitive. Verify if you copy your work from a local computer to your home directory on ARC.

**Run the cell below to submit your work.**

- You may submit your work multipe times up to the deadline of the assignment.
- Please note that any files that you previously submitted will
be overwritten by your current files.

In [ ]:
! ./submit.sh -y